# Regridding AOD:
    
This notebook will generate the results in the AOD column on Table 1, as well as Figure 2, 3 and 4, the data needed to get the results in Tables 3 and 7.


First, we import all relevant libraries.

In [ ]:
import sys
import os
import netCDF4 as ntf
from pyhdf.SD import SD, SDC
import numpy as np
import math
from netCDF4 import Dataset 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy
import xesmf as xmf
import xarray as xr
import glob

Here, as there was no simple tool provided with the data, we use a script to download the data, which we include as a function and its function call.

In [ ]:
def download_aod_data():
    s1 = "https://portal.nccs.nasa.gov/datashare/maiac/DataRelease/CMG_0.05degree/AOT5km/2015/MAIACCMG.2015"
    s3 = ".hdf"

    for i in range(day_start,day_end):
        outname = LOCAL + str(year) + "_" + str(i) + ".hdf"

        #Uncomment this section if you are running through your downloaded files and checking they are the right size.
        '''
        redownload_mode = True
        size = os.path.getsize(outname)/1000000.0
        print(str(size) + " MB")
        if(size != expected_size_mb):
            print("Re-downloading needed")
        else:
            continue
        '''
        
        progress_str = "Downloading Day " + str(i) + " of " + str(day_end - day_start)
        if(redownload_mode):
            progress_str = "Re-downloading Day " + str(i)

        print(progress_str)      
        s2 = str(i)
        if(i < 10):
            s2 = "00" + str(i)
        if(i >= 10 and i < 100):
            s2 = "0" + str(i)
        link = s1+s2+s3

        cmd = "curl -o " + outname + " " + link
        os.system(cmd)

In [ ]:
# Download data if not already downloaded
download_aod_data()

Here, we set up our parameters including info about the desired region, the size of the target grid, filepaths, latitude stride for the regridding, selected days, scale factor to apply to the data, and the key threshold to use for marking a target box as missing or not. Regular mode considers averaging non-missing cells for target, binary mode simply reports 1s or 0s. 

In [ ]:
LOCAL = "/data0/rm3873/regrid_data_005/"
day_start = 1
day_end = 366
year = 2015
binary_path = '/data0/rm3873/binary_regridded_v2.nc'
land_mask_path = '/data0/zzheng/GEOS-Chem-grid/land_mask.nc'
TARGET_MISSING = -9999
WATER = -4999
lat_st = 6
lat_end = 36.25
lat_siz = 0.25
lon_st = 68.125
lon_end = 97.8126
lon_siz = 0.3125
target_grid_lats = 121
target_grid_lons = 96
orig_grid_size = 0.05
SCALE_FACTOR = 0.01
regular_mode = False

We manually extract the desired dataset and limit it othe region of interest.

In [ ]:
data = []
MISSING = -28672
for day in range(day_start, day_end):
    fname = LOCAL + str(year) + "_" + str(day) + ".hdf"
    print(fname)
    f = SD(fname, SDC.READ)
    sel = f.select('AOT').get()
    data.append(np.array(sel))

In [ ]:
data = np.array(data)
data_ = []
for day in range(365):
    print(day)
    cur = data[day]
    cur_ = []
    for i in range(1080,1680):
        lons = []
        for j in range(4960,5560):
            lons.append(cur[i][j])
        cur_.append(lons)
    data_.append(cur_)
data_ = np.array(data_)
print(np.shape(data_))

Let's check the percent of missing data in the raw grid.

In [ ]:
np.count_nonzero(data_ == MISSING)/(365*600*600)*100 

.... and get the empty new grid.

In [ ]:
regridded = np.full((day_end-day_start,target_grid_lats,target_grid_lons), TARGET_MISSING)

General purpose diophantine equation solver, we determined the appropriate lat/long box stride lengths but could work on general solver to automate and observe

In [ ]:
def simple_diophantine_solver():
    # ax + by + cz + ... = orig_lat_size, x+y+z=new_lat_size
    # dq + er + fs + ... = orig_lon_size, q+r+s=new_lon_size
    pass
    

Let's setup the regridded NetCDF file with our desired dimensions, we will fill in the 'aot' variable.

In [ ]:
land_mask = Dataset(land_mask_path,mode='r',format='NETCDF4_CLASSIC')
ncfile = Dataset(binary_path,mode='w',format='NETCDF4_CLASSIC') 
lat_dim = ncfile.createDimension('lat', target_grid_lats)     
lon_dim = ncfile.createDimension('lon', target_grid_lons)
time = ncfile.createDimension('time',day_end-day_start)

lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'days of 2015'
time.long_name = 'days_of_the_year'
# Define a 3D variable to hold the data
aot = ncfile.createVariable('aot',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost

lat[:] = np.arange(lat_st,lat_end,lat_siz)
lon[:] = np.arange(lon_st,lon_end,lon_siz)
time[:] = np.arange(day_start,day_end)

Here, we get the crux of the regridding. We map the box of cells in the source to a target cell, by checking if the number of present cells is above our threshold. If it is, we average the value of the present source cell to get the target cell, else we mark it as missing.

In [ ]:
lat_start_idx = int((90 - lat_end)/orig_grid_size) 
lat_end_idx = int((90 - lat_st)/orig_grid_size) 
lon_start_idx = int((180 + lon_st)/orig_grid_size)
lon_end_idx = int((180 + lon_end)/orig_grid_size) 

for day in range(day_start,day_end):
    
    #Part I: Read
    print('Day: ' + str(day))
    fname = LOCAL + str(year) + "_" + str(day) + ".hdf"
    f = SD(fname, SDC.READ)
    sel = f.select('AOT').get()
    MISSING = -28672
    data = np.full((3600,7200), MISSING)
    for m in range(3600):
        for n in range(7200):
            data[m][n] = sel[m][n]
    
    #Part II
    i=lat_start_idx
    ct_lat = 0
    lat_str = 5
    threshold = 0.5


    while(i < lat_end_idx):
        #Part III
        j = lon_start_idx
        ct_lon = 0
        lon_str = 6
        while(j < lon_end_idx):
            if(ct_lon == 90):
                lon_str = 9

            if(land_mask["mask"][ct_lat][ct_lon] == 0):
                regridded[day - 1][ct_lat][ct_lon] = WATER
                ct_lon = ct_lon + 1
                j = j + lon_str
                continue

            #Part IV
            #print('Bounding box: [' + str(i) + ',' + str(i+lat_str) + ' by ' + str(j) + ',' + str(j+lon_str) + '] for idx (' + str(ct_lat) + ',' + str(ct_lon) + ')')
            cur_row = [m for m in range(i,i+lat_str)] 
            cur_col = [n for n in range(j,j+lon_str)]
            cur_box_idx = np.ix_(cur_row,cur_col)
            cur_box = data[cur_box_idx]

            #Part V
            total_ct = len(cur_box) * len(cur_box[0])
            non_miss = []
            for a in range(len(cur_box)):
                for b in range(len(cur_box[a])):
                    if(cur_box[a][b] != MISSING):
                        non_miss.append(cur_box[a][b]* SCALE_FACTOR)

            #Part VI
            cur_val = TARGET_MISSING
            if(not regular_mode):
                cur_val = 0
            if(len(non_miss) >= threshold * total_ct):
                non_miss = np.array(non_miss)
                cur_val = np.average(non_miss)
                if(not regular_mode):
                    cur_val = 1

            #Part VII
            regridded[day - 1][ct_lat][ct_lon] = cur_val
            ct_lon = ct_lon + 1
            j = j + lon_str


        ct_lat = ct_lat + 1
        i = i + lat_str
aot[::] = regridded
ncfile.close()

Let's read in the simulated data (already matched for the GEOS-Chem target grid), and copy them into a dictionary of the species

In [ ]:
raw_emission = xr.open_dataset("/data0/rm3873/dsi_india/daily_emission.nc").sel(lat=slice(6,36),lon=slice(68,98))
raw_gas = xr.open_dataset("/data0/rm3873/dsi_india/daily_gas_column.nc").sel(lat=slice(6,36),lon=slice(68,98))
raw_pm = xr.open_dataset("/data0/rm3873/dsi_india/daily_surface_pm25_RH50.nc").sel(lat=slice(6,36),lon=slice(68,98))
raw_met = xr.open_dataset("/data0/rm3873/dsi_india/daily_meteo.nc").sel(lat=slice(6,36),lon=slice(68,98))
raw_aod = xr.open_dataset("/data0/rm3873/dsi_india/daily_aod.nc").sel(lat=slice(6,36),lon=slice(68,98))
raw_emission["EmisDST_Natural"] = raw_emission["EmisDST1_Natural"] + raw_emission["EmisDST2_Natural"] + raw_emission["EmisDST3_Natural"] + raw_emission["EmisDST4_Natural"]
feature_ml = [
    {"PM25":[]},
    {'CO_trop':[], 'SO2_trop':[], 'NO2_trop':[], 'CH2O_trop':[], 'NH3_trop':[]},
     {'AOT_C':[], 'AOT_DUST_C':[]},
    {'T2M':[], 'PBLH':[], 'U10M':[], 'V10M':[], 'PRECTOT':[], 'RH':[]},
    {'EmisDST_Natural':[], 
                'EmisNO_Fert':[], 'EmisNO_Lightning':[], 'EmisNO_Ship':[], 'EmisNO_Soil':[]}]
sets = [raw_pm,raw_gas,raw_aod,raw_met,raw_emission]

for i in range(len(sets)):
    for spec in feature_ml[i]:
        print(spec)
        cur_set = sets[i][spec]
        
        for entry in cur_set:
            feature_ml[i][spec].append([])
        for j in range(len(cur_set)):
            print(j)
            for k in range(len(cur_set[j])):
                feature_ml[i][spec][j].append(cur_set[j][k])
        feature_ml[i][spec] = np.array(feature_ml[i][spec])

Now we can simply apply our AOD regridded missing mask onto all these datasets!

In [ ]:
missing_list = np.argwhere(np.isnan(regridded))
missing_vals = [np.NaN] * len(missing_list)
for j in range(len(sets)):
    for spec in feature_ml[j]:
        print(spec)
        feature_ml[j][spec][tuple(np.transpose(missing_list))] = missing_vals

Finally, let's write all these datasets, now with the AOD missing mask applied to them, to disk!

In [ ]:
for i in range(len(feature_ml)):
    for spec in feature_ml[i]:
        fname = '/data0/rm3873/custom_regridded_aod_' + str(spec) + '.nc'
        ncfile = Dataset(fname,mode='w',format='NETCDF4_CLASSIC') 
        lat_dim = ncfile.createDimension('lat', 121)     
        lon_dim = ncfile.createDimension('lon', 96)
        time = ncfile.createDimension('time',365)

        lat = ncfile.createVariable('lat', np.float32, ('lat',))
        lat.units = 'degrees_north'
        lat.long_name = 'latitude'
        lon = ncfile.createVariable('lon', np.float32, ('lon',))
        lon.units = 'degrees_east'
        lon.long_name = 'longitude'
        time = ncfile.createVariable('time', np.float64, ('time',))
        time.units = 'days of 2015'
        time.long_name = 'days_of_the_year'
        # Define a 3D variable to hold the data
        key_var = ncfile.createVariable(spec,np.float64,('time','lat','lon'))

        lat[:] = np.arange(6,36.25,0.25)
        lon[:] = np.arange(68.125,97.8126,0.3125)
        time[:] = np.arange(1,366)
        key_var[::] = feature_ml[i][spec]
        ncfile.close()